In [48]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
from random import randint

In [49]:
options = webdriver.ChromeOptions() 
options.add_argument("download.default_directory=" + os.getcwd() + "/downloads")

In [50]:
start_page = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="
test_name = "can alpaslan"
print(start_page)

https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG=


In [51]:
#Use pandas create dataframe (df) object that will store the data
columns = ['name','paper_type','citation','url','abstract','title','collaborators']
citations = pd.DataFrame()
#Place the list of names generated previously in the project into the df object
citations

""


In [52]:
#df1['e'] = Series(np.random.randn(sLength), index=df1.index)
#citations['a'] = pd.Series(test_name, index=citations.index)


Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

In [53]:
#def google_scholar(url):
    
google_scholar = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="

#Instatiate the webdriver and set to implicitly wait for page objects to load
driver = webdriver.Chrome()
driver.implicitly_wait(10) # seconds loading implicitly

#Set driver to go to start_page and search a given name with valid scholar profile
driver.get(google_scholar)
print(driver.current_url)
search_bar = driver.find_element_by_class_name('gs_in_txt')
search_bar.send_keys(test_name)
search_bar.send_keys(Keys.ENTER)


#Set driver to click the first profile given as a search result. Will need to be modified to be more general later.
view_profile_link = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3/a')
view_profile_link.click()


#pull the first citation for the page
view_citation_link = driver.find_element_by_class_name('gsc_a_at')
view_citation_link.click()

#Citation information title, publication date, journal, volume, issue, pages, publisher, abstract.

citations = ['Professor','Authors','Publication_Title', 'URL','Citation']



citation_table = driver.find_element_by_id("gsc_vcd_table")
citation_table_data_objects= citation_table.find_elements_by_class_name("gs_scl")

citation = {
    "Authors" : "",
    "Title" : "",
    "Date" : "",
    "Publisher": ""
}

for data in citation_table_data_objects:
    
    data_category = data.find_element_by_tag_name("div")
    
    print(data_category.text)
    
    if data_category.text == "Authors":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        print(data_field.text)
        citation["Authors"] = data_field.text
    elif data_category.text == "Publication date":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Date"] = data_field.text
    elif data_category.text == "Publisher":
        data_field = data.find_element_by_class_name("gsc_vcd_value")
        citation["Publisher"] = data_field.text


# citation_table_data = [data.text for data in citation_table_data_objects]
# for data in citation_table_data:
#     print(data)

# if citation_table.find_elements_by_xpath('//div/div')
# pub_title = driver.find_element_by_class_name('gsc_vcd_title_link')
# pub_title = pub_title.text


# pub_date = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[2]/div[2]')
# pub_date = pub_date.text
# pub_journal = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[3]/div[2]')
# pub_journal = pub_journal.text
# pub_vol = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[4]/div[2]')
# pub_vol = pub_vol.text
# pub_iss = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[5]/div[2]')
# pub_iss = pub_iss.text
# pub_pgs = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[6]/div[2]')
# pub_pgs = pub_pgs.text
# pub_publisher = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[7]/div[2]')
# pub_publisher = pub_publisher.text
# pub_abstr = driver.find_element_by_xpath('//*[@id="gsc_vcd_descr"]/div/div')
# pub_abstr = pub_abstr.text 
# print(pub_title)
# print(pub_date)
# print(pub_journal)
# print(pub_vol)
# print(pub_iss)
# print(pub_pgs)
# print(pub_publisher)
# print(pub_abstr)

# Retrieve all the urls of the first page
#research_a_tags = driver.find_elements_by_xpath("html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[2]/div/a")
#urls = [tag.get_attribute("href") for tag in research_a_tags]
#titles = [tag.text for tag in research_a_tags]
#paper_types = driver.find_elements_by_xpath('html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[3]/div/div/span')
#paper_types = [item.text for item in paper_types]
#num_research_items -= 100

#current_url = driver.current_url

#page_count = 2
#while num_research_items > 0:
    #driver.get(current_url + '/' + str(page_count))
    #temp_tags = driver.find_elements_by_xpath("html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[2]/div/a")
    #for tag in temp_tags:
        #urls.append(tag.get_attribute("href"))
        #titles.append(tag.text)
    #temp_papers = driver.find_elements_by_xpath('html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[3]/div/div/span')
    #for paper in temp_papers:
        #paper_types.append(paper.text)
    #num_research_items -= 100
    #page_count += 1

#while "Full-text available" in paper_types:
    #paper_types.remove("Full-text available")
driver.close()
    

https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG=
Authors
Ian I Mitroff, Murat C Alpaslan
Publication date
Publisher
Description
Total citations
Scholar articles


In [47]:
print(citation['Authors'])
print(citation['Date'])
print(citation['Publisher'])

Ian I Mitroff, Murat C Alpaslan
2003/4/1
Harvard Business School Pub.


In [53]:
count = 0
#for url in urls:
driver = webdriver.Chrome(options=options)
#time.sleep(randint(5,10))

driver.get(urls[0])

# At this point we already have name, title, type, url
# Need citation, collabs, date, DOI, publisher, edition, series, volume, issue, ISSN, ISBN, handle, book title, 

if paper_types[0] == 'Article':
    # this should get the DOI and Abstract in one go
    
    publication_details = driver.find_elements_by_class_name('publication-details__section')
    print(publication_details)
    
    data = publication_details[0].find_elements_by_xpath('//div/div[2]')
    
    print(data)
    
    #meta = driver.find_elements_by_xpath('html/body/div[2]/main/section/section/div/div[2]')
    #print(meta)
    #doi = meta[0].text.split("")
    #doi = doi[1]
    #abstract = meta[1].text
    # then we'll get the collaborators
    
    
    # Attempt to get the date
    #date = driver.find_element_by_xpath('html/body/div[2]/main/section/section/div/div/span[2]')
    
    #download the citatation
    
    # take it apart, perhaps improve it
    
    # build a pandas row
    
    # add it to the main dataframe
    # end loop
    
#print(doi)
#print(abstract)

    
#elif paper_type[0] == "Conference Paper"





driver.close()
    #count +=1

NameError: name 'urls' is not defined

In [99]:
counter = 0
for url in urls:
    print(counter, ":", url)
    counter +=1

0 : https://www.researchgate.net/publication/315416439_Composite_mechanics_of_the_multilayer_structure_of_the_annulus_fibrosus
1 : https://www.researchgate.net/publication/313626183_Modeling_and_Analysis_of_Heart_Left_Ventricle_in_Diastole_Using_Finite_Element
2 : https://www.researchgate.net/publication/285611966_Machine_design_Redesigned
3 : https://www.researchgate.net/publication/261035675_Endoprosthetic_reconstruction_after_resection_of_musculoskeletal
4 : https://www.researchgate.net/publication/40768832_Cemented_Distal_Femoral_Endoprostheses_for_Musculoskeletal_Tumor_Improved_Survival_of_Modular_versus_Custom_Implants
5 : https://www.researchgate.net/publication/44571522_How_Long_Do_Endoprosthetic_Reconstructions_for_Proximal_Femoral_Tumors_Last
6 : https://www.researchgate.net/publication/44624077_Cemented_Endoprosthetic_Reconstruction_of_the_Proximal_Tibia_How_Long_Do_They_Last
7 : https://www.researchgate.net/publication/51381639_Hysteretic_pinching_of_human_secondary_osteons

Note that ResearchGate only loads 100 citations per page, if number of citations % 100 = 0 we go to second page

In [22]:
citation = {
    "Authors" : "",
    "Title" : ""
}

In [25]:
citation['Authors'] = 'Jorge Martinez'

In [26]:
citation['Authors']

'Jorge Martinez'

In [24]:
citation['Title']

''